# A slightly less naive and foolish keras model

(Not working yet)


In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(123)

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.layers.recurrent import LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import TimeDistributed
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from tqdm import tqdm

from preprocess import *
from util import *

In [21]:
#TODO: shift data down to 0-89 instead of 19-...
NUM_POSSIBLE_ACTIONS = len(pitchToTuple(SILENCE))
EMBEDDING_DIMENSIONS=12
#MIN_SONG_LENGTH=200 #TODO figure out/look up

In [34]:
NUM_SONGS = 900

sop = loadData('Soprano')[:NUM_SONGS]
min_song_length = min(len(song) for song in sop)

data = trimmed_sop = np.array([song[:min_song_length] for song in sop])

bass = loadData('Bass')[:NUM_SONGS]
trimmed_categorical_bass = [to_categorical(song[:min_song_length], nb_classes=NUM_POSSIBLE_ACTIONS) for song in sop]
labels = np.array(trimmed_categorical_bass)


In [35]:


model = Sequential()

model.add(Embedding(NUM_POSSIBLE_ACTIONS, EMBEDDING_DIMENSIONS, input_length=min_song_length))

model.add(LSTM(EMBEDDING_DIMENSIONS, return_sequences=True))

#model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(NUM_POSSIBLE_ACTIONS)))

print(model.output_shape)
model.add(Activation('softmax'))

# Binary crossentropy?
model.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['accuracy'])



(None, 82, 89)


In [36]:


model.fit(data, labels, nb_epoch=10, batch_size=32)

Epoch 1/10
900/900 [==============================] - 1s - loss: 0.0111 - acc: 0.3720     
Epoch 2/10
900/900 [==============================] - 0s - loss: 0.0108 - acc: 0.4573     
Epoch 3/10
900/900 [==============================] - 0s - loss: 0.0104 - acc: 0.4573     
Epoch 4/10
900/900 [==============================] - 0s - loss: 0.0096 - acc: 0.4573     
Epoch 5/10
900/900 [==============================] - 0s - loss: 0.0087 - acc: 0.4573     
Epoch 6/10
900/900 [==============================] - 0s - loss: 0.0083 - acc: 0.4573     
Epoch 7/10
900/900 [==============================] - 0s - loss: 0.0079 - acc: 0.4573     
Epoch 8/10
900/900 [==============================] - 0s - loss: 0.0074 - acc: 0.4573     
Epoch 9/10
900/900 [==============================] - 0s - loss: 0.0069 - acc: 0.4573     
Epoch 10/10
900/900 [==============================] - 0s - loss: 0.0064 - acc: 0.4573     


In [40]:
output=model.predict(data[0, np.newaxis])
print(output.shape)
np.array([np.argmax(output[0, time, :]) for time in range(output.shape[1])])+LOWEST_PITCH

(1, 82, 89)


IndexError: index 82 is out of bounds for axis 1 with size 82